In [4]:
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Dense, Activation, Dropout
import pandas as pd
import numpy as np

### Load the training and test data and extract the labels 

In [5]:
train_df = pd.read_csv("train.csv")

In [6]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
train_df['label'].dtype

dtype('int64')

In [15]:
labels = train_df.iloc[:, 0].values.astype('int32')

In [29]:
train_X = train_df.iloc[:, 1:].values.astype('float32')
train_X.shape

(42000, 784)

In [20]:
labels

array([1, 0, 1, ..., 7, 6, 9], dtype=int32)

In [25]:
labels.shape

(42000,)

In [22]:
train_Y = np_utils.to_categorical(labels)

In [23]:
train_Y

array([[ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [24]:
train_Y.shape

(42000, 10)

In [30]:
test_X = pd.read_csv("test.csv").values.astype('float32')
test_X.shape

(28000, 784)

### Scale the trainning and test data by dividing the max of the training data and subtracting the mean (after dividing) of the training data.

In [32]:
scale = np.max(train_X)

In [33]:
train_X = train_X / scale
test_X = test_X / scale

In [34]:
mean = np.mean(train_X)
train_X -= mean
test_X -= mean

### Get the dimensions of the respective layers

In [36]:
input_dim = train_X.shape[1]
output_dim = train_Y.shape[1]
input_dim, output_dim

(784, 10)

### Build the MLP (Multiple Layered Perceptron)

In [37]:
model = Sequential()

<b> Input Layer:</b>

In [39]:
model.add(Dense(128, input_dim = input_dim))

In [40]:
model.add(Activation('relu'))

In [41]:
model.add(Dropout(0.15))

<b>Second Layer:</b>

In [42]:
model.add(Dense(128))

In [43]:
model.add(Activation('relu'))

In [44]:
model.add(Dropout(0.15))

<b>Output Layer:</b>

In [45]:
model.add(Dense(output_dim))

In [46]:
model.add(Activation('softmax'))

### Add loss function and optimizer

In [47]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

### Train and predict 

In [51]:
print("trainining....")
model.fit(train_X, train_Y, epochs=10, batch_size=16, validation_split=0.1, verbose=2)

trainining....
Train on 37800 samples, validate on 4200 samples
Epoch 1/10
7s - loss: 0.3143 - val_loss: 0.1963
Epoch 2/10
7s - loss: 0.1849 - val_loss: 0.1458
Epoch 3/10
7s - loss: 0.1596 - val_loss: 0.1647
Epoch 4/10
7s - loss: 0.1516 - val_loss: 0.1418
Epoch 5/10
8s - loss: 0.1483 - val_loss: 0.1619
Epoch 6/10
8s - loss: 0.1507 - val_loss: 0.1601
Epoch 7/10
8s - loss: 0.1546 - val_loss: 0.2053
Epoch 8/10
8s - loss: 0.1523 - val_loss: 0.1751
Epoch 9/10
8s - loss: 0.1535 - val_loss: 0.1750
Epoch 10/10
8s - loss: 0.1613 - val_loss: 0.1977


In [52]:
print("Generating test predictions....")
preds = model.predict_classes(test_X, verbose=0)

Generating test predictions....


#### Write to a CSV file

In [57]:
def write_file(preds, filename):
    pd.DataFrame({"ImageId":list(range(1, len(preds) + 1)), "Label":preds}).to_csv(filename, \
                                                                                   index=False,
                                                                                   header=True)

In [58]:
write_file(preds, "MLP_submissions.csv")